In [ ]:
from glob import glob
from os.path import join
import os
import numpy as np
import itertools
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, DotProduct, WhiteKernel
import matplotlib.pyplot as plt
import pandas as pd

# Preprocessed data reading

In [ ]:
data_folder = join('..', 'data', 'preprocessed_data')

In [ ]:
data_files = glob(join(data_folder, '*.csv'))

In [ ]:
timeseries = []

for f in data_files:
    timeseries.append(pd.read_csv(f, index_col=0))

# ML pipeline

In [ ]:
pca = PCA()

In [ ]:
gpr = GaussianProcessRegressor()

In [ ]:
rbf_kernel = RBF()

In [ ]:
dot_kernel = DotProduct() + WhiteKernel()

In [ ]:
pipe = Pipeline(steps=[('pca', pca), ('gpr', gpr)])

## Parameter tuning

### list algorithms parameters

In [ ]:
print(pca.get_params().keys())
print(gpr.get_params().keys())

### list kernel parameters

In [ ]:
print(rbf_kernel.get_params().keys())
print(dot_kernel.get_params().keys())

### generate configurations to test

In [ ]:
config1 = {'pca__n_components': [1, 2, 3],
           'gpr__kernel': [rbf_kernel],
           'gpr__kernel__length_scale': [0.1, 1, 2, 10]}

In [ ]:
config2 = {'pca__n_components': [1, 2, 3],
           'gpr__kernel': [dot_kernel],
           'gpr__kernel__k1__sigma_0': [0.1, 1, 2, 10],
           'gpr__kernel__k2__noise_level': [0.01, 0.1, 1, 2, 10]}

In [ ]:
tuned_parameters = [config1, config2]

In [ ]:
search = GridSearchCV(pipe, tuned_parameters, n_jobs=-1)

# Test single timeserie

## Data extraction

In [ ]:
x_header = ['ExactoKnife_' + axis for axis in ['x', 'y', 'z']]

In [ ]:
y_header = ['opt_force_' + axis for axis in ['x', 'y', 'z']]

In [ ]:
X = timeseries[0][x_header]

In [ ]:
y1 = timeseries[0][y_header] # multiple dimensions force

In [ ]:
y2 = np.linalg.norm(y1.values,axis=1) # single dimension normed force

## Best parameter search

In [ ]:
search.fit(X, y1)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
search.fit(X, y2)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)